In [1]:
from utils.dataloader import *
from utils.sift import *
from utils.helper import *
from utils.classifier import get_report, down_sampling, vote, up_sampling, one_vs_other_up_sampling, distance_vote
import pickle


In [2]:
RANDOME_STATE= 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
train_collection = test_collection = y_train = y_test = None


with open(os.path.join(OUT_PATH, 'sift_X.pkl'), 'rb') as f:
    (train_collection, test_collection) = pickle.load(f)


with open(os.path.join(OUT_PATH, 'sift_y.pkl'), 'rb') as f:
    (y_train, y_test) = pickle.load(f)


In [4]:

ks = train_collection.keys()
y_train = np.array(y_train)
y_test = np.array(y_test)

for a, b in zip(train_collection.values(), test_collection.values()):
    print(a.shape, b.shape)

print(ks)
print(y_train.shape, y_test.shape)

(1968, 128) (656, 128)
(1968, 256) (656, 256)
(1968, 512) (656, 512)
(1968, 1024) (656, 1024)
dict_keys([128, 256, 512, 1024])
(1968,) (656,)


In [5]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

param = {
    'kernel': ['linear', 'rbf'],
    'C' : [0.01, 0.1, 1, 10, 100]
}


clf_collections = []
predictions_collections = []
votes_collections = []


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import shuffle

# Create the data points for each class
class_A = np.random.rand(100, 2) + np.array([1.0, 1.0])
class_B = np.random.rand(100, 2) + np.array([2.0, 2.0])
class_C = np.random.rand(100, 2) + np.array([3.0, 3.0])

# Combine all class data points
X = np.concatenate([class_A, class_B, class_C])

# Create labels for each class
y = np.concatenate([np.zeros(len(class_A)), np.ones(len(class_B)), np.ones(len(class_C))*2])

# Shuffle the dataset
X, y = shuffle(X, y, random_state=0)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
predictions = []

total_classes = np.unique(y_train)
k = 1
clfs = one_vs_other_up_sampling(total_classes, X_train, y_train,
                                clf = SVC,
                                param=param, k=k, logger=None,
                                knn = KNeighborsClassifier, n_neighbors=2)


# distance, pred= distance_vote(X_test=X_test, clfs=clfs, predictions=predictions)

distance , pred = distance_vote(X_test, clfs, None)

76 164
Generated 152 synthetic samples.
Class: [-1.  0.] Val Score: 1.00 use {'C': 10, 'kernel': 'linear'}
Positive: (76,), Negative: (164,)
81 159
Generated 81 synthetic samples.
Class: [-1.  1.] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Positive: (81,), Negative: (159,)
83 157
Generated 83 synthetic samples.
Class: [-1.  2.] Val Score: 1.00 use {'C': 1, 'kernel': 'linear'}
Positive: (83,), Negative: (157,)


In [8]:
get_report(y_test, pred)

Testset Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        24
         1.0       1.00      1.00      1.00        19
         2.0       1.00      1.00      1.00        17

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60




(1.0,
 array([[24,  0,  0],
        [ 0, 19,  0],
        [ 0,  0, 17]]))